# Data Munging: Strings

Data munging, the process of wrestling with data to make it into something clean and usable, is an important part of any job analyzing data.

Today we're going to focus on some data that has information we want, but the information is not properly *structured*. In particular, it comes as a single column with a string value, and we want to turn it into a series of boolean columns.

To do that, we're going to use the powerful built-in methods Python provides us to work with strings. You can read all about the available methods here: 

https://docs.python.org/3/library/string.html

In particular, we're going to use `.split()`, which is a method that turns a string into a list of strings, and `.strip()`, which removes the "whitespace" from a string.

In [1]:
# Play:
#
# Take a look at the official Python documentation for the
# "split" and "strip" methods. Play around with them now
# to make sure you understand how they work:



In [2]:
#
# 1) 
# Read the data in a csv called "jobs.csv" into a DataFrame.
# This data is from a site that posts job ads online. 
# Each row represents an ad for a job on the site.

import pandas as pd

df = pd.read_csv('jobs.csv')

In [3]:
# 
# Take a look at your data and note that you have
# a column called `pay`. That column is a string,
# as far as Python is concerned. However, to us
# humans, we notice that the information is more
# structured than that. It seems like a "collection
# of keywords," where each job can have zero or more
# keywords such as "Part-Time" or "Contract" which
# describe the type of contract.
#
# There are 6 different contract types. 
# 
# Your goal:
# Transform the DataFrame, adding 6 boolean columns, 
# one for each contract type, indicating whether or
# not that job has that contract type.
#
# NOTE: This is a relatively large task. 
# Break it down into a series of steps, just like
# we did in the last exercises. Work on each
# step separately.
#
# Many of the steps will require to work with the
# string methods mentioned above. 

In [4]:
#
# 2)
# Break down your tasks, write a "pipeline" function
# called "add_contract_types".
#
# HINT: last time, each "step" returned a DataFrame
# object. This might not be the case this time, the
# steps can return any data type that is helpful
# to move the to next step!


def add_contract_types(df):
    
    # If I had an iterable of strings I 
    # could split each string on a given
    # separator.
    pay_lists = split_strings(df.pay, ',')

    # If I had a the `pay` from each row
    # as a list of strings, I could flatten the list
    contract_types = flatten(pay_lists)

    # If I had a giant list of all the types
    # i could distinct that list to get unique types
    contract_types = get_unique_types(contract_types)

    # if I had the unique types, I could
    # check if each one is the pay
    # and add a column
    df = add_columns(df, contract_types)

    return df

In [5]:
#
# 3) 
# Now write all the "steps" (functions) needed
# by your pipeline function (add_contract_types)


def safe_split(s, sep):
    try:
        return [word.strip() for word in s.split(sep)] 
    except AttributeError:
        return []

def split_strings(col, sep):
    return [safe_split(s, sep) for s in col]

def flatten(li):
    return [y for x in li for y in x]

def get_unique_types(cols):
    return pd.Series(cols).unique()

def get_keyword(keyword, s):
    try:
        return keyword in s
    except TypeError:
        return False

def add_columns(df, cols):
    df = df.copy()
    for col in cols:
        df[col] = df.pay.map(lambda s: get_keyword(col, s))
    return df

In [6]:
# 
# 4)
# Now add the needed columns by using your function
# add_contract_types. You will want the returned
# DataFrame for some of the further exercises.

df = add_contract_types(df)

df.head()

,company,pay,category,sub_category,Temporary,Internship,Contract,Part-time,Commission,Other
0,1-800-Flowers.com,NaN,Computer/internet,Web Producer jobs,False,False,False,False,False,False
1,1-800-Flowers.com,NaN,Computer/internet,VP of Information Technology jobs,False,False,False,False,False,False
2,1-800-Flowers.com,NaN,Construction/facilities,Yard Supervisor jobs,False,False,False,False,False,False
3,1-800-Flowers.com,NaN,Construction/facilities,Yard Person jobs,False,False,False,False,False,False
4,1-800-Flowers.com,NaN,Construction/facilities,Yard Manager jobs,False,False,False,False,False,False


In [7]:
#
# 5) 
# Assume that all jobs that don't specify a contract
# type in "pay" are Full-time. Create a new column, 
# called "Full-time", which is a boolean that 
# should be True if the job is Full-time, false otherwise.

df['Full-time'] = df.pay.isna()

In [8]:
#
# 6)
# Get the percentage of jobs for each contract type
# i.e. number of jobs of X type / number of jobs

(df.iloc[:, -7:].sum() / df.shape[0])

Temporary     0.032832
Internship    0.011767
Contract      0.030639
Part-time     0.114479
Commission    0.009634
Other         0.001666
Full-time     0.817808
dtype: float64

In [9]:
# 
# 7)
# Which industries ('category') have the highest
# percentage of part-time jobs posted?
# The lowest?

def percent_type(contract, df):
    return df[contract].mean()

df.groupby('category') \
  .apply(lambda df: percent_type('Part-time', df)) \
  .sort_values()

category
Computer/internet                0.025357
Upper Management/consulting      0.032432
Marketing/advertising/pr         0.034346
Manufacturing/mechanical         0.050317
Accounting/finance               0.056391
Insurance                        0.063665
Arts/entertainment/publishing    0.071248
Construction/facilities          0.079808
Human Resources                  0.081542
Banking/loans                    0.083533
Pharmaceutical/bio-tech          0.098169
Telecommunications               0.101351
Legal                            0.107579
Administrative                   0.120390
Transportation/logistics         0.132564
Customer Service                 0.145433
Real Estate                      0.156912
Education/training               0.157357
Healthcare                       0.193516
Government/military              0.213247
Hospitality/travel               0.216232
Non-profit/volunteering          0.248191
Restaurant/food Service          0.250936
Law Enforcement/security 

In [10]:
#
# 8)
# Which industries ('category') have the highest
# percentage of Internship jobs posted?
# The lowest?

# Note: this question is very similar to the last.
# make a function that can answer both questions

def by_contract_type(contract, df):
    return df.groupby('category') \
             .apply(lambda df: percent_type(contract, df)) \
             .sort_values()

by_contract_type('Temporary', df)

category
Banking/loans                    0.008216
Upper Management/consulting      0.010811
Computer/internet                0.013320
Marketing/advertising/pr         0.016513
Real Estate                      0.021171
Human Resources                  0.025574
Accounting/finance               0.025898
Insurance                        0.026915
Healthcare                       0.028276
Construction/facilities          0.029202
Hospitality/travel               0.029512
Pharmaceutical/bio-tech          0.030010
Administrative                   0.030754
Customer Service                 0.030849
Transportation/logistics         0.031871
Telecommunications               0.033784
Restaurant/food Service          0.036517
Legal                            0.041565
Law Enforcement/security         0.048414
Non-profit/volunteering          0.056205
Manufacturing/mechanical         0.057439
Education/training               0.058410
Arts/entertainment/publishing    0.062381
Government/military      

In [11]:
#
# 9)
# Use your function to ask the same question about
# Comission jobs

by_contract_type('Commission', df)

category
Restaurant/food Service          0.000936
Arts/entertainment/publishing    0.001900
Non-profit/volunteering          0.003339
Computer/internet                0.003611
Pharmaceutical/bio-tech          0.004578
Construction/facilities          0.005309
Real Estate                      0.007472
Manufacturing/mechanical         0.007741
Transportation/logistics         0.008776
Government/military              0.008885
Law Enforcement/security         0.008904
Hospitality/travel               0.009081
Marketing/advertising/pr         0.009247
Education/training               0.009256
Upper Management/consulting      0.010811
Administrative                   0.011001
Accounting/finance               0.011278
Legal                            0.012225
Banking/loans                    0.012325
Customer Service                 0.012821
Healthcare                       0.013237
Human Resources                  0.016679
Insurance                        0.025362
Telecommunications       

In [12]:
#
# 10)
# Let's call jobs that are either Temporary, 
# Part-time or Internships "precarious". 
#
# Order the industries (category) by the 
# percentage of precarious jobs
#
# HINT: can you modify some previous function
# to make this question easy to answer?
#
# HINT: Make sure your variables reflect their
# content. Collections should be plural, single
# elements should be singular.

def percent_type(contracts, df):
    return df[contracts].mean().sum()

def by_contract_type(contracts, df):
    return df.groupby('category') \
             .apply(lambda df: percent_type(contracts, df)) \
             .sort_values()

by_contract_type(['Part-time', 'Internship', 'Temporary'], df)

category
Computer/internet                0.055529
Upper Management/consulting      0.063063
Marketing/advertising/pr         0.070674
Insurance                        0.092650
Banking/loans                    0.098254
Accounting/finance               0.099833
Human Resources                  0.115271
Construction/facilities          0.118799
Manufacturing/mechanical         0.119678
Telecommunications               0.140766
Pharmaceutical/bio-tech          0.144964
Administrative                   0.160895
Transportation/logistics         0.169515
Legal                            0.176039
Arts/entertainment/publishing    0.183344
Customer Service                 0.183894
Real Estate                      0.186800
Education/training               0.219917
Healthcare                       0.226745
Hospitality/travel               0.254824
Restaurant/food Service          0.292447
Non-profit/volunteering          0.308848
Law Enforcement/security         0.317752
Government/military      

In [13]:
#
# 11)
# Get the 5 companies who post the most jobs
# in each  category, along with the number of 
# jobs listed by each company.

def get_popular_company(df):
    return df.groupby('company') \
             .size() \
             .sort_values(ascending=False) \
             .reset_index(name='count') \
             .iloc[:5]


df.groupby('category').apply(get_popular_company)

company  count
category                                                   
Accounting/finance          0        JP Morgan Chase     74
                            1                   Citi     36
                            2        Bank of America     32
                            3            Wells Fargo     26
                            4                 Kyriba     21
...                                              ...    ...
Upper Management/consulting 0        JP Morgan Chase     29
                            1  UNITED PARCEL SERVICE      9
                            2           Avera Health      8
                            3                Verizon      7
                            4                   MUFG      6

[120 rows x 2 columns]

In [14]:
# 12)
# Is any company in the top 5 across more than one categories??
# Return the companies who are, along with the categories
# in which they appear in the top 5.
#
# FORMAT: Dataframe with 3 columns: company, category, number of jobs
#
# HINT: take a look at the `.filter` method on GroupBy:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.DataFrameGroupBy.filter.html


df.groupby('category') \
  .apply(get_popular_company) \
  .reset_index(level=0) \
  .groupby('company') \
  .filter(lambda df: df.shape[0] > 1) \
  .sort_values(['company', 'count'], ascending=False) \
  [['company', 'category', 'count']]

,company,category,count
2,Wells Fargo,Banking/loans,115
2,Wells Fargo,Human Resources,40
3,Wells Fargo,Accounting/finance,26
2,Verizon,Telecommunications,30
3,Verizon,Upper Management/consulting,7
2,VCA Animal Hospitals,Healthcare,119
4,VCA Animal Hospitals,Hospitality/travel,45
3,US Department of the Army,Administrative,57
4,US Department of the Army,Government/military,23
1,US Department of Veterans Affairs,Healthcare,159
